# Model selected: SVM + TFIDF
### By **Néstor Suat** in 2020

**Descripción:** Generando el modelo entrenado con los parametros seleccionados del modelo de aprendizaje automatico **SVM** con la técnica de embedding **TFIDF**.

**Input:**
* Train and Test set
* Parameters selected

**Output:**
* Model output (SVM + TFIDF)

***

### Importando librerías

Como estamos en un archivo afuera se necesita agregar la dirección ../ (raíz del proyexto) para importar la librería de preprocesamiento.

In [1]:
import pandas as pd
import numpy as np

import pickle

import sys
sys.path.insert(0, '../../../')

from classes.tweet2accident.preprocessing import Preprocessing

### Importando datasets

In [2]:
train = pd.read_csv("../../../data/v1/7030/train70.tsv", delimiter = "\t", quoting = 3)
test = pd.read_csv("../../../data/v1/7030/test30.tsv", delimiter = "\t", quoting = 3)
dataset = pd.concat([train,test]) 

print(train.shape, test.shape, dataset.shape) # (3804, 3)
print(train.shape[0] + test.shape[0], dataset.shape[0])

(2662, 2) (1142, 2) (3804, 2)
3804 3804


In [3]:
X, y = dataset.text, dataset.label
X_train, y_train = train.text, train.label
X_test, y_test = test.text, test.label

## 1) Preprocessing and SVM Model Parameters

In [4]:
type_clean = 5 # The type clean in this case 5 corresponding to 5_steam

#TFIDF
max_df = 0.5
max_features = 800
min_df = 0.001
ngram_range= (1, 1)

#SVM
C=4
gamma=0.7
kernel= 'rbf'

## 2) Model

### Libraries

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

### Support Vector Machine (**SVM**) Model

Creación de pipeline del modelo seleccionado: **5_steam + TFIDF + SVM**

In [6]:
pipeline = Pipeline([
    ('transform', Preprocessing(type_clean=type_clean, njobs=4)),
    ('tfidf', TfidfVectorizer(ngram_range=ngram_range, max_df=max_df, min_df=min_df, max_features=max_features)),
    ('clf', SVC(random_state=123, kernel=kernel, C=C, gamma=gamma))
])

In [7]:
pipeline.fit(X, y)

Pipeline(memory=None,
         steps=[('transform', Preprocessing(njobs=4, type_clean=5)),
                ('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.5, max_features=800,
                                 min_df=0.001, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 SVC(C=4, cache_size=200, class_weight=None, coef0=0.0,
   

## 3) Evaluation

### Evaluación con **Train/test**
La partición de los datos para el conjunto de entrenamiento y pruebas consta de una relación de 70/30 correspondientemente.

In [8]:
# Predicting the Test set results
y_pred = pipeline.predict(X_test)

# Making the Confusion Matrix
cm_svm = confusion_matrix(y_test, y_pred)

metrics_svm = []
metrics = {}
metrics['accuracy'] = accuracy_score(y_test, y_pred)
metrics['recall'] = recall_score(y_test, y_pred)
metrics['precision'] = precision_score(y_test, y_pred)
metrics['f1'] = f1_score(y_test, y_pred)
metrics_svm.append(metrics)
metrics_svm = pd.DataFrame(metrics_svm)

In [9]:
print(metrics_svm)
print(cm_svm)

   accuracy    recall  precision       f1
0  0.999124  0.998221        1.0  0.99911
[[580   0]
 [  1 561]]


## 4) Save model with Pickel

Save the model to disk

In [10]:
filename = 'model_m2_tfidf_svm.pkl'
with open(filename, 'wb') as model_file:
      pickle.dump(pipeline, model_file)

Load the model from disk

In [11]:
accident_clasification_model = pickle.load(open(filename, 'rb'))

In [12]:
#Probando el modelo de clasificación cargado
text_predict = accident_clasification_model.predict(["choque entre camion y carro particular"])
print(text_predict[0]) #El resultado es 1 si es accidente, y 0 si no lo es.

1


In [13]:
text_test = [
    "choque entre camion y carro particular",
    "caí por accidente en tu corazón", 
    "accidente aereo deja 100 muertos en australia",
    "hola mundo",
    "accidente carro"
]
text_predict = accident_clasification_model.predict(text_test)
print(text_predict) #El resultado es 1 si es accidente, y 0 si no lo es.

[1 0 0 0 0]


In [ ]:
df = pd.DataFrame(data=text_test,columns=["text"])

In [ ]:
df["label"] = text_predict

In [ ]:
df